# Homework 5 - Text Labeling

* Peyman Naseri, `96100522`
* Mobina Pournemat, `97105833`
* Mahsa Amani, `97105769`

# setting gpu

for use local gpu run this comand in anaconda prompet and copy url and paste on colab:

    pip install jupyter_http_over_ws
    jupyter serverextension enable --py jupyter_http_over_ws
    jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0

https://research.google.com/colaboratory/local-runtimes.html  


In [ ]:
import torch
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
      # tf.config.experimental.set_virtual_device_configuration(gpu,
      #                                                         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*6)])

  except RuntimeError as e:
    print(e)


if len(tf.config.list_physical_devices('GPU')) > 0:
  !nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla T4, 460.32.03, 15109 MiB


# requirement package

<div dir=rtl>
ابتدا کتابخانه های مورد نیاز را نصب می کنیم.
</div>

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
%%capture
!pip install transformers
!pip install hazm
!pip install seqeval
!pip install datasets

# DATA

## Load data

<div dir="rtl">
در ابتدا داده های مورد نظر را لود می کنیم و سپس به سه بخش train و evaluation و test تقسیم می کنیم. در ادامه نمایی از داده ها را نیز مشاهده می کنید.
</div>

In [3]:
import json
import pandas as pd


with open('dataset_annotated_splited.json', encoding="utf8") as f:
    data = json.load(f)

In [4]:
labels = ['train','eval', 'test']
train_len, eval_len, test_len = [len(data[l]) for l in labels]

In [5]:
print("Training size: ", train_len)
print("Validation size: ", eval_len)
print("Test size: ", test_len)

Training size:  1350
Validation size:  75
Test size:  75


In [6]:
i = 2
print(f"Sample training data {i}...")
print("Header:")
print(data["train"][i]["header"])
print("Header Labels:")
print(data["train"][i]["annotations"][1]["header"])
print("Text:")
print(data["train"][i]["text"])
print("Text Labels:")
print(data["train"][i]["annotations"][1]["text"])

Sample training data 2...
Header:
دمشق: غرب از آوارگان سوری برای اهداف سیاسی خود استفاده می کند
Header Labels:
[{'name': 'LOC', 'range': [0, 4]}, {'name': 'LOC', 'range': [6, 9]}]
Text:
به گزارش گروه بین الملل خبرگزاری تسنیم به نقل از خبرگزاری رسمی سوریه سانا ، «فیصل المقداد» وزیر امور خارجه سوریه گفت که کشورش تلاش های زیادی برای بازگشت آوارگان انجام می دهد و از آنها می خواهد بدون هیچ قید و شرطی به کشور خود بازگردند. وی در جمع خبرنگاران پس از پایان نشست گفت وگو درباره جنگ رسانه ای در سوریه در چارچوب برنامه نشست مشترک سوریه و روسیه، گفت: اقدامات اجباری یکجانبه مانع بازگشت آوارگان می شود و کشورهای غربی نمی خواهند آوارگان سوری به وطن خود برگردند تا از آنان در اعمال فشارهای سیاسی علیه سوریه سوء استفاده کنند
Text Labels:
[{'name': 'ORG', 'range': [24, 38]}, {'name': 'ORG', 'range': [49, 73]}, {'name': 'PER', 'range': [77, 89]}, {'name': 'LOC', 'range': [107, 112]}, {'name': 'LOC', 'range': [303, 308]}, {'name': 'LOC', 'range': [337, 342]}, {'name': 'LOC', 'range': [345, 350]}, {'name': 'LOC

<div dir="rtl">
با استفاده از تابع زیر، همه ی برچسب گذاری های جملات را با هم اجتماع می گیریم.
</div>

In [7]:
def union(d):
  l ={'header':[], 'text':[]}
  for info in d['annotations']:
    for anot in info['header']:
      if anot not in l['header']:
        l['header'].append(anot)
    for anot in info['text']:
      if anot not in l['text']:
        l['text'].append(anot)
  d['annotations'] = l

for data_name in labels:
  for d in data[data_name]:
    union(d)

## <p style="background-color:skyblue; font-family:newtimeroman; font-size:150%; text-align:center; border-radius: 15px 50px;">Data Pre-processing 🛠</p>

Now we are going to engineering the data to make it easier for the model to clasiffy.

This section is very important to reduce the dimensions of the problem.

In [8]:
from __future__ import unicode_literals
from hazm import *
import re
import string
import codecs

## Tokenizer

### Bert_tokenizer

<div dir="rtl">
در ادامه از مدل از پیش آموزش دیده شده را از HooshvareLab لود می کنیم تا در ادامه از آن استفاده کنیم.
</div>

In [9]:
from transformers import BertTokenizer
from transformers import AutoModelForTokenClassification  # for pytorch
from transformers import pipeline

model_name_or_path = "HooshvareLab/bert-fa-zwnj-base-ner" 

In [10]:
tokenizer = BertTokenizer.from_pretrained(model_name_or_path)

<div dir="rtl">

</div>

<div dir="rtl">
در ادامه برای تمیز کردن بیشتر داده ها، کاراکتر های اضافی را با "فاصله" جایگزین می کنیم و سپس tokenization انجام می دهیم و در دیتافریم ذخیره می کنیم
</div>

In [11]:
def replace_chars(s):
    chars = ["/", "-", "|", "+", "*", "\u200c", "?"]
    for c in chars:
        s = s.replace(c, " ")
    s = s.replace(u'\xa0', u' ')
    return s

In [12]:
names_data = ['train', 'eval', 'test']
names_text = ['header', 'text']

def tokenization(name_data):

  def f(name_data, i, name_text):
    sent = replace_chars(data[name_data][i][name_text])
    sent = data[name_data][i][name_text]
    out_tokens = [w.strip() for w in word_tokenize(sent)]
    out_tags = ["O" for i in out_tokens]
    for tag in data[name_data][i]["annotations"][name_text]:
        s, e = tag["range"]
        tokens = word_tokenize(sent[s:e])
        for token in tokens:
            try:
                index = out_tokens.index(token)
                out_tags[index] = tag["name"]
            except Exception as e:
                pass
    return out_tokens, out_tags

  df = pd.DataFrame(columns=['tokens', 'ner_tags'])

  for i in range(len(data[name_data])):
    header_tokens, header_tags = f(name_data, i, 'header')
    text_tokens, text_tags = f(name_data, i, 'text')
    df.loc[i] = [header_tokens+text_tokens, header_tags+text_tags]

  return df

df_train, df_eval, df_test = (tokenization(d) for d in names_data)

In [13]:
df_train

,tokens,ner_tags
0,"[«, صندلی, داغ, », و, «, شب, آرام, », در, نورو...","[O, O, O, O, O, O, O, O, O, O, DAT, DAT, O, O,..."
1,"[نشان, درجه, یک, هنری, استاد, «, محمدرضا, اسحا...","[O, O, O, O, O, O, PER, PER, O, O, O, O, O, O,..."
2,"[دمشق, :, غرب, از, آوارگان, سوری, برای, اهداف,...","[LOC, LOC, LOC, O, O, NAT, O, O, O, O, O, O, O..."
3,"[از, این, همه, تناقض, اذیت, نمی, شوید, ؟!, /, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[WHO, منشاء, جدیدی, برای, ویروس, کرونای, جدید,...","[ORG, O, O, O, O, O, O, O, O, O, O, ORG, ORG, ..."
...,...,...
1345,"[انتشار, شماره, جدید, «, چراغ, نامه, », به, گز...","[EVE, EVE, EVE, EVE, EVE, EVE, O, O, O, ORG, O..."
1346,"[میزان, بهینه, بنزین, سهمیه, ای, به, هر, فرد, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1347,"[ضرورت, اختصاص, بودجه, بیشتر, به, انجمن, های, ...","[O, O, O, O, O, ORG, ORG, O, O, O, O, O, O, O,..."
1348,"[آیا, حمله, روسیه, به, اوکراین, آغاز, فروپاشی,...","[O, EVE, EVE, EVE, EVE, O, EVE, EVE, EVE, EVE,..."


<div dir="rtl">
در ادامه نیز از نرمالایزر استفاده می کنیم تا دیتا را برای دادن به مدل آماده کنیم.
</div>

In [14]:
normalizer = Normalizer()

def clean_text(text):
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~«»؟!،'''
    for ele in text:
        if ele in punc:
            text = text.replace(ele, "")
    return text
 
stemmer = Stemmer()

# Persian Stopwords
# https://github.com/sobhe/hazm/blob/master/hazm/data/stopwords.dat
stop_words = [normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]


def preprocess_data(text):
    text = text.strip()
    text = normalizer.normalize(text)
    text = clean_text(text)
    text = text.strip()
    return text

In [15]:
def data_cleaner(row):
  n = len(row['tokens'])
  for i in range(n-1,-1,-1):
    token = row['tokens'][i]
    clean_token = preprocess_data(token)
    if clean_token =='':
      row['tokens'].pop(i)
      row['ner_tags'].pop(i)
    else:
      row['tokens'][i] = clean_token


def df_cleaner(df):
  gf = df.copy()
  gf.apply(lambda x: data_cleaner(x), axis=1)
  return gf
df = df_cleaner(df_train)

In [16]:
df_train, df_eval, df_test = df_cleaner(df_train), df_cleaner(df_eval),df_cleaner(df_test)

In [17]:
df_train

,tokens,ner_tags
0,"[صندلی, داغ, و, شب, آرام, در, نوروزی, های, راد...","[O, O, O, O, O, O, DAT, DAT, O, O, ORG, O, ORG..."
1,"[نشان, درجه, یک, هنری, استاد, محمدرضا, اسحاقی,...","[O, O, O, O, O, PER, PER, O, O, O, O, O, O, O,..."
2,"[دمشق, غرب, از, آوارگان, سوری, برای, اهداف, سی...","[LOC, LOC, O, O, NAT, O, O, O, O, O, O, O, O, ..."
3,"[از, این, همه, تناقض, اذیت, نمی, شوید, از, تلا...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[WHO, منشاء, جدیدی, برای, ویروس, کرونای, جدید,...","[ORG, O, O, O, O, O, O, O, O, O, O, ORG, ORG, ..."
...,...,...
1345,"[انتشار, شماره, جدید, چراغ, نامه, به, گزارش, ا...","[EVE, EVE, EVE, EVE, EVE, O, O, ORG, O, O, O, ..."
1346,"[میزان, بهینه, بنزین, سهمیه, ای, به, هر, فرد, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1347,"[ضرورت, اختصاص, بودجه, بیشتر, به, انجمن, های, ...","[O, O, O, O, O, ORG, ORG, O, O, O, O, O, O, O,..."
1348,"[آیا, حمله, روسیه, به, اوکراین, آغاز, فروپاشی,...","[O, EVE, EVE, EVE, EVE, O, EVE, EVE, EVE, EVE,..."


<div dir="rtl">
در این بخش دیتافریم ها را به دیتاست تبدیل کرده و برچسب ها را به عدد نگاشت می کنیم .
</div>

In [18]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(df_train)
eval_dataset = Dataset.from_pandas(df_eval)
test_dataset = Dataset.from_pandas(df_test)

In [19]:
label_encoding_dict = {"PER": 0, "LOC": 1, "mainLOC": 2, "ORG": 3, "EVE": 4, "NAT": 5, "TIM": 6, "DAT": 7, "O": 8}

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

In [21]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)
    
    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [22]:
train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [23]:
eval_tokenized_datasets = eval_dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

# BERT

## Pretrained Model

<div dir="rtl">
حالا مدل از پیش آموزش دیده شده را لود می کنیم و تعداد برچسب ها را نیز ست می کنیم.
</div>

In [25]:
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import EarlyStoppingCallback


model = AutoModelForTokenClassification.from_pretrained(
    model_name_or_path, num_labels=9, ignore_mismatched_sizes=True)

# model = AutoModelForTokenClassification.from_pretrained(
#     model_name_or_path)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base-ner and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([21, 768]) in the checkpoint and torch.Size([9, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([21]) in the checkpoint and torch.Size([9]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Training

<div dir="rtl">
سپس آرگومان های training را ست کرده و از early stopping برای جلوگیری از ضعیف شدن مدل پس از تعدادی epoch استفاده می کنیم که معیار مورد استفاده آن جهت تعیین خوب بودن مدل ، f1 ست می شود.
</div>

In [26]:
batch_size = 16

args = TrainingArguments(
    f"test-ner",
    evaluation_strategy = "steps",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=1e-5,
    load_best_model_at_end = True,
    metric_for_best_model = 'f1',
)

data_collator = DataCollatorForTokenClassification(tokenizer)

In [30]:
# !pip install nervaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<div dir="rtl">
در این بخش  تابع compute_metrics را که از کتابخانه ی seqeval استفاده می کند را تعریف می کنیم. هم چنین برای به دست آوردن دقت بهتر از کتابخانه ی nervaluate استفاده کردیم ولی نتیجه مطلوبی حاصل نشد.
</div>

In [27]:
import numpy as np
from datasets import load_metric
# from sklearn.metrics import classification_report
from seqeval.metrics import classification_report
# import tqdm
from tqdm.notebook import tqdm
# from nervaluate import Evaluator


label_list = ["PER", "LOC", "mainLOC", "ORG", "EVE", "NAT", "TIM", "DAT", "O"]

metric = load_metric("seqeval")

# def report_result_classification(preds, labels):
#   '''
#   report result precision, recall, f1-score on test data
#   '''
#   preds_flat = np.argmax(preds, axis=1).flatten()
#   labels_flat = labels.flatten()
#   r = classification_report(preds_flat, labels_flat)
#   return r

results_by_tag = None
results = None

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    # evaluator = Evaluator(true_labels, true_predictions, tags=label_list[:-1], loader="list")
    # results, results_by_tag = evaluator.evaluate()
    # return results_by_tag
    # r = report_result_classification(true_predictions, true_labels)
    r = classification_report(true_predictions, true_labels)
    tqdm.write(f'\n{r}')
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}


<div dir="rtl">
در این بخش برای اینکه از loss function  وزن دار استفاده کنیم کلاس CustomTrainer را تعریف می کنیم و به برچسب های مختلف وزن های گوناگونی میدهیم و وزن برچسب O را از همه کمتر میدهیم. (به دلیل imbalanced بودن برچسب ها)
</div>

In [30]:
import torch
from torch import nn
from transformers import Trainer


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 40.0, 1.0]).to('cuda'))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

<div dir="rtl">
در این مرحله نیز ترین کردن مدل را شروع می کنیم.
</div>

In [33]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=eval_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
)

trainer.train()
trainer.evaluate()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: __index_level_0__, tokens, ner_tags. If __index_level_0__, tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 1350
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 850


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.021000,0.591642,0.456193,0.422378,0.438635,0.911085


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: __index_level_0__, tokens, ner_tags. If __index_level_0__, tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 75
  Batch size = 16



              precision    recall  f1-score   support

          AT       0.50      0.47      0.48       109
          ER       0.52      0.55      0.53       180
          IM       0.22      0.28      0.24        18
          OC       0.42      0.32      0.36       131
          VE       0.37      0.48      0.42       198
      ainLOC       0.24      0.35      0.28        26

   micro avg       0.42      0.46      0.44       662
   macro avg       0.38      0.41      0.39       662
weighted avg       0.43      0.46      0.44       662



Saving model checkpoint to test-ner/checkpoint-500
Configuration saved in test-ner/checkpoint-500/config.json
Model weights saved in test-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from test-ner/checkpoint-500 (score: 0.438634713144517).
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: __index_level_0__, tokens, ner_tags. If __index_level_0__, tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 75
  Batch size = 16



              precision    recall  f1-score   support

          AT       0.50      0.47      0.48       109
          ER       0.52      0.55      0.53       180
          IM       0.22      0.28      0.24        18
          OC       0.42      0.32      0.36       131
          VE       0.37      0.48      0.42       198
      ainLOC       0.24      0.35      0.28        26

   micro avg       0.42      0.46      0.44       662
   macro avg       0.38      0.41      0.39       662
weighted avg       0.43      0.46      0.44       662



{'epoch': 10.0,
 'eval_accuracy': 0.9110849270370706,
 'eval_f1': 0.438634713144517,
 'eval_loss': 0.5916416645050049,
 'eval_precision': 0.4561933534743202,
 'eval_recall': 0.4223776223776224,
 'eval_runtime': 1.6151,
 'eval_samples_per_second': 46.437,
 'eval_steps_per_second': 3.096}

## Test

<div dir="rtl">
در نهایت نیز مدل را بر روی داده ی تست، می آزماییم و نتایج را بررسی می کنیم
</div>

In [34]:
predictions = trainer.predict(test_tokenized_datasets)

The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: __index_level_0__, tokens, ner_tags. If __index_level_0__, tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 75
  Batch size = 16



              precision    recall  f1-score   support

          AT       0.40      0.44      0.42       111
          ER       0.37      0.45      0.41       173
          IM       0.12      0.20      0.15        15
          OC       0.38      0.34      0.36       116
          VE       0.45      0.50      0.48       266
      ainLOC       0.25      0.25      0.25        28

   micro avg       0.40      0.44      0.42       709
   macro avg       0.33      0.36      0.34       709
weighted avg       0.40      0.44      0.42       709



# References

* [BERT model](https://huggingface.co/HooshvareLab/bert-fa-zwnj-base-ner)

* [Fine tunning](https://medium.com/@andrewmarmon/fine-tuned-named-entity-recognition-with-hugging-face-bert-d51d4cb3d7b5)